In [1]:
import pandas as pd
import csv
import re
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
import gensim
from gensim import corpora, models
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import time
from pprint import pprint

stemmer = SnowballStemmer("english") # the stopwords 
def remain_english(line):
    rule =re.compile("[^a-z^A-Z]")
    line = rule.sub('',line)
    return line
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v')) # remain problems

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(str(text)) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
#             result.append(lemmatize_stemming(token))
            result.append(remain_english(lemmatize_stemming((token))).lower()) # take out not english and return into verb, and lower it 
    return result

C:\Users\B_Areej\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\B_Areej\Anaconda3\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [4]:
# read file into pandas
import pandas as pd

fileGP = r'GamesReview.csv'
fileApple = r'apple-games-reviews.csv'

df1 = pd.read_csv(fileGP)
df2 = pd.read_csv(fileApple)
# take the target subcate reviews (Apple and GP)
dfAdventureGP = df1[df1.appSubGener.str.contains('Adventure')] # the Adventure games from GP
dfStrategyGP = df1[df1.appSubGener.str.contains('Strategy')] # the Strategy games from GP

dfAdventureAp = df2[df2.genres.str.contains('Adventure')] # the Adventure games from Apple 
dfStrategyAp = df2[df2.genres.str.contains('Strategy')] # the Strategy games from Apple 

In [7]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\B_Areej\AppData\Roaming\nltk_data...


True

# Adventure part

In [8]:
# analyse Adventure part
df1 = dfAdventureGP
df2 = dfAdventureAp
gpNum = dfAdventureGP.count()['text']
ApNum = dfAdventureAp.count()['text'] # text(reviews) number from apple
print('targe subcate reviews num of GP and Apple:',gpNum,ApNum)

sid = SentimentIntensityAnalyzer()
text_num = gpNum + ApNum
documentNeg=[]
documentPos=[]
inconsistantNum1 = 0 # inconsistant reviews number in low rating reviews
inconsistantNum2 = 0
for i in df1.score.index:
    if(df1.score[i]<3): # low rating reviews
#         print(sid.polarity_scores(df1.text[i])['neg'])
        if(sid.polarity_scores(str(df1.text[i]))['pos'] > 0.5): # 如果是这条评论的sensitive分析分数显示这条评论其实是负面的,threhold = 0.5
            inconsistantNum1 += 1
        else:
            documentNeg.append(df1.text[i])             

    if(df1.score[i]>3): # high rating reviews
        if(sid.polarity_scores(str(df1.text[i]))['neg'] > 0.5): # if the neg score is high, not use this review,threhold = 0.5
            inconsistantNum2 += 1
        else:
            documentPos.append(df1.text[i]) 
            
for j in df2.score.index:
    if(df2.score[j]<3): # low rating reviews
        if(sid.polarity_scores(str(df2.text[j]))['pos'] > 0.5): # 如果是这条评论的sensitive分析分数显示这条评论其实是负面的,threhold = 0.5
            inconsistantNum1 += 1
        else:
            documentNeg.append(df2.text[j])             
    if(df2.score[j]>3): # low rating reviews
        if(sid.polarity_scores(str(df2.text[j]))['neg'] > 0.5): # if the neg score is high, not use this review,threhold = 0.5
            inconsistantNum2 += 1
        else:
            documentPos.append(df2.text[j])      
                        
print('number of reviews text:',text_num)
print('number of neg:',len(documentNeg))
print('number of inconsistant neg reviews text(actually a pos reviews):',inconsistantNum1)
print('number of pos:',len(documentPos))
print('number of inconsistant pos reviews text(actually an neg reviews):',inconsistantNum2)



targe subcate reviews num of GP and Apple: 7862 208041
number of reviews text: 215903
number of neg: 41834
number of inconsistant neg reviews text(actually a pos reviews): 200
number of pos: 149281
number of inconsistant pos reviews text(actually an neg reviews): 366


In [9]:
# negtive LDA  reviews analyse
document = documentNeg
doc_clean = [preprocess(doc) for doc in document]

dictionary = corpora.Dictionary(doc_clean)

# 使用上面的词典，将转换文档列表（语料）变成 DT 矩阵
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
print('made doc_term_matrix')

Lda = gensim.models.ldamodel.LdaModel
print('object num',len(doc_term_matrix))
start = time.time()
lda_model = gensim.models.LdaMulticore(doc_term_matrix, num_topics=10, id2word=dictionary, passes=100, workers=4)
# result = lda_model.print_topics(num_words=10)
end = time.time()
print ('Fitting Time:',end-start)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
# print(result)

made doc_term_matrix
object num 41834
Fitting Time: 1126.0647239685059
Topic: 0 
Words: 0.035*"control" + 0.034*"game" + 0.021*"kill" + 0.013*"like" + 0.013*"shoot" + 0.013*"jump" + 0.011*"button" + 0.011*"hard" + 0.011*"time" + 0.010*"zombi"
Topic: 1 
Words: 0.047*"game" + 0.036*"play" + 0.034*"energi" + 0.029*"time" + 0.018*"wait" + 0.018*"item" + 0.017*"take" + 0.017*"need" + 0.015*"like" + 0.015*"quest"
Topic: 2 
Words: 0.075*"game" + 0.025*"like" + 0.013*"stori" + 0.012*"review" + 0.012*"star" + 0.011*"good" + 0.011*"play" + 0.010*"look" + 0.010*"charact" + 0.009*"think"
Topic: 3 
Words: 0.076*"game" + 0.037*"play" + 0.036*"peopl" + 0.023*"like" + 0.022*"world" + 0.018*"think" + 0.017*"know" + 0.017*"want" + 0.014*"friend" + 0.012*"say"
Topic: 4 
Words: 0.127*"game" + 0.029*"stupid" + 0.029*"play" + 0.029*"like" + 0.028*"time" + 0.027*"wast" + 0.025*"hate" + 0.021*"download" + 0.016*"minecraft" + 0.016*"worst"
Topic: 5 
Words: 0.142*"level" + 0.022*"battl" + 0.019*"beat" + 0.015*"

In [10]:
# positive LDA reviews analyse
document = documentPos
doc_clean = [preprocess(doc) for doc in document]

dictionary = corpora.Dictionary(doc_clean)
# 使用上面的词典，将转换文档列表（语料）变成 DT 矩阵
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
Lda = gensim.models.ldamodel.LdaModel
print('object num',len(doc_term_matrix))
start = time.time()
lda_model = gensim.models.LdaMulticore(doc_term_matrix, num_topics=10, id2word=dictionary, passes=100, workers=4)
# result = lda_model.print_topics(num_words=10)
end = time.time()
print ('Fitting Time:',end-start)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
# print(result)

object num 149281
Fitting Time: 4577.42399430275
Topic: 0 
Words: 0.101*"game" + 0.095*"level" + 0.044*"good" + 0.033*"time" + 0.026*"challeng" + 0.026*"hard" + 0.022*"easi" + 0.019*"pretti" + 0.019*"play" + 0.018*"great"
Topic: 1 
Words: 0.047*"game" + 0.041*"peopl" + 0.029*"like" + 0.018*"friend" + 0.016*"thing" + 0.016*"play" + 0.016*"player" + 0.015*"think" + 0.015*"know" + 0.014*"good"
Topic: 2 
Words: 0.087*"game" + 0.023*"stori" + 0.019*"great" + 0.018*"graphic" + 0.018*"play" + 0.012*"gameplay" + 0.010*"charact" + 0.009*"amaz" + 0.009*"control" + 0.009*"worth"
Topic: 3 
Words: 0.061*"game" + 0.039*"money" + 0.025*"time" + 0.024*"play" + 0.024*"spend" + 0.020*"like" + 0.020*"energi" + 0.017*"thing" + 0.015*"need" + 0.014*"coin"
Topic: 4 
Words: 0.055*"like" + 0.044*"love" + 0.030*"game" + 0.018*"anim" + 0.017*"think" + 0.016*"thing" + 0.014*"cute" + 0.012*"charact" + 0.011*"hors" + 0.011*"dragon"
Topic: 5 
Words: 0.046*"game" + 0.036*"like" + 0.020*"weapon" + 0.020*"control" + 0

# The Strategy Part

In [11]:
# analyse strategy part
df1 = dfStrategyGP
df2 = dfStrategyAp
gpNum = dfStrategyGP.count()['text']
ApNum = dfStrategyAp.count()['text'] # text(reviews) number from apple
print('targe subcate reviews num of GP and Apple:',gpNum,ApNum)

sid = SentimentIntensityAnalyzer()
text_num = gpNum + ApNum
documentNeg=[]
documentPos=[]
inconsistantNum1 = 0 # inconsistant reviews number in low rating reviews
inconsistantNum2 = 0
for i in df1.score.index:
    if(df1.score[i]<3): # low rating reviews
#         print(sid.polarity_scores(df1.text[i])['neg'])
        if(sid.polarity_scores(str(df1.text[i]))['pos'] > 0.5): # 如果是这条评论的sensitive分析分数显示这条评论其实是负面的,threhold = 0.5
            inconsistantNum1 += 1
        else:
            documentNeg.append(df1.text[i])             

    if(df1.score[i]>3): # high rating reviews
        if(sid.polarity_scores(str(df1.text[i]))['neg'] > 0.5): # if the neg score is high, not use this review,threhold = 0.5
            inconsistantNum2 += 1
        else:
            documentPos.append(df1.text[i]) 
            
for j in df2.score.index:
    if(df2.score[j]<3): # low rating reviews
        if(sid.polarity_scores(str(df2.text[j]))['pos'] > 0.5): # 如果是这条评论的sensitive分析分数显示这条评论其实是负面的,threhold = 0.5
            inconsistantNum1 += 1
        else:
            documentNeg.append(df2.text[j])             
    if(df2.score[j]>3): # low rating reviews
        if(sid.polarity_scores(str(df2.text[j]))['neg'] > 0.5): # if the neg score is high, not use this review,threhold = 0.5
            inconsistantNum2 += 1
        else:
            documentPos.append(df2.text[j])      
                        
print('number of reviews text:',text_num)
print('number of neg:',len(documentNeg))
print('number of inconsistant neg reviews text(actually a pos reviews):',inconsistantNum1)
print('number of pos:',len(documentPos))
print('number of inconsistant pos reviews text(actually an neg reviews):',inconsistantNum2)


targe subcate reviews num of GP and Apple: 12371 197903
number of reviews text: 210274
number of neg: 46501
number of inconsistant neg reviews text(actually a pos reviews): 237
number of pos: 142298
number of inconsistant pos reviews text(actually an neg reviews): 304


In [12]:
# negtive LDA  reviews analyse
document = documentNeg
doc_clean = [preprocess(doc) for doc in document]

dictionary = corpora.Dictionary(doc_clean)

# 使用上面的词典，将转换文档列表（语料）变成 DT 矩阵
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
print('made doc_term_matrix')

Lda = gensim.models.ldamodel.LdaModel
print('object num',len(doc_term_matrix))
start = time.time()
lda_model = gensim.models.LdaMulticore(doc_term_matrix, num_topics=10, id2word=dictionary, passes=100, workers=4)
# result = lda_model.print_topics(num_words=10)
end = time.time()
print ('Fitting Time:',end-start)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
# print(result)

made doc_term_matrix
object num 46501
Fitting Time: 2095.3590557575226
Topic: 0 
Words: 0.040*"game" + 0.031*"version" + 0.023*"play" + 0.015*"updat" + 0.014*"card" + 0.011*"screen" + 0.010*"button" + 0.010*"like" + 0.009*"turn" + 0.008*"control"
Topic: 1 
Words: 0.060*"level" + 0.040*"game" + 0.017*"attack" + 0.017*"player" + 0.017*"play" + 0.016*"time" + 0.015*"lose" + 0.014*"card" + 0.011*"match" + 0.011*"troop"
Topic: 2 
Words: 0.043*"game" + 0.027*"support" + 0.025*"custom" + 0.019*"issu" + 0.015*"develop" + 0.012*"respons" + 0.012*"problem" + 0.012*"servic" + 0.011*"help" + 0.010*"player"
Topic: 3 
Words: 0.082*"game" + 0.022*"like" + 0.020*"play" + 0.014*"time" + 0.013*"wait" + 0.010*"bore" + 0.010*"thing" + 0.009*"clan" + 0.009*"" + 0.009*"clash"
Topic: 4 
Words: 0.047*"roll" + 0.023*"time" + 0.022*"random" + 0.022*"play" + 0.019*"piec" + 0.018*"dice" + 0.017*"need" + 0.017*"doubl" + 0.016*"cheat" + 0.015*"black"
Topic: 5 
Words: 0.137*"game" + 0.040*"like" + 0.031*"play" + 0.0

In [13]:
# positive LDA reviews analyse
document = documentPos
doc_clean = [preprocess(doc) for doc in document]

dictionary = corpora.Dictionary(doc_clean)
# 使用上面的词典，将转换文档列表（语料）变成 DT 矩阵
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
Lda = gensim.models.ldamodel.LdaModel
print('object num',len(doc_term_matrix))
start = time.time()
lda_model = gensim.models.LdaMulticore(doc_term_matrix, num_topics=10, id2word=dictionary, passes=100, workers=4)
# result = lda_model.print_topics(num_words=10)
end = time.time()
print ('Fitting Time:',end-start)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
# print(result)

object num 142298
Fitting Time: 3527.2574694156647
Topic: 0 
Words: 0.091*"level" + 0.022*"hard" + 0.021*"beat" + 0.017*"time" + 0.017*"challeng" + 0.015*"game" + 0.012*"button" + 0.012*"get" + 0.011*"need" + 0.011*"easi"
Topic: 1 
Words: 0.110*"game" + 0.063*"play" + 0.036*"updat" + 0.021*"love" + 0.020*"time" + 0.015*"great" + 0.013*"work" + 0.012*"crash" + 0.012*"version" + 0.011*"phone"
Topic: 2 
Words: 0.153*"game" + 0.044*"great" + 0.042*"play" + 0.032*"good" + 0.018*"graphic" + 0.016*"like" + 0.015*"enjoy" + 0.013*"easi" + 0.012*"strategi" + 0.012*"recommend"
Topic: 3 
Words: 0.209*"game" + 0.155*"love" + 0.069*"play" + 0.055*"addict" + 0.043*"awesom" + 0.033*"best" + 0.017*"amaz" + 0.017*"like" + 0.014*"" + 0.012*"super"
Topic: 4 
Words: 0.039*"build" + 0.036*"battl" + 0.033*"like" + 0.032*"attack" + 0.025*"base" + 0.025*"clan" + 0.024*"troop" + 0.023*"unit" + 0.022*"clash" + 0.016*"citi"
Topic: 5 
Words: 0.043*"player" + 0.021*"card" + 0.014*"play" + 0.013*"game" + 0.013*"peop